In [ ]:
import matplotlib.patches
import numpy as np
import seaborn as sns
from IPython.display import display
from ipywidgets import interact
from jupyter_renderer_widget import PyplotRenderer
from matplotlib import cm
from matplotlib import pyplot as plt
from scipy.signal import convolve

In [ ]:
def get_circular_kernel(width, height=None):
    if height is None:
        height = width
    x_vals = np.linspace(-1, 1, width)
    y_vals = np.linspace(-1, 1, height)
    x_grid, y_grid = np.meshgrid(x_vals, y_vals)
    z_grid = x_grid ** 2 + y_grid ** 2
    mask = (z_grid > 0.7) & (z_grid <= 1)
    z_grid = np.where(mask, 1., 0.)
    z_grid /= z_grid.sum()
    return z_grid


def render(ax, time, threshold=0.21, do_threshold=True):
    mat = np.zeros((60, 60))
    mat[22, 7] = 1.
    mat[38, 29] = 1.
    mat[24, 27] = 1.

    iterations = int(time) + 1
    interp = time % 1.

    kernel = kernel1 * 10
    prev_mat = np.zeros_like(mat)
    for i in range(iterations):
        prev_mat = mat
        mat = convolve(mat, kernel, 'same')
        kernel = kernel2
        if do_threshold:
            mat = np.where(mat > threshold, 1., 0.)
            
    mat = mat * interp + prev_mat * (1 - interp)
    ax.imshow(mat, vmin=0, vmax=1, cmap='bone')

    
kernel1 = get_circular_kernel(15)
kernel2 = get_circular_kernel(7)


if False:
    @interact(
        time=(0., 100., 0.1),
        threshold=(0., 1., 0.01),
    )
    def run(
        time=0.,
        threshold=0.21,
        do_threshold=True,
    ):
        fig = plt.figure(figsize=(14, 10))
        ax = fig.gca()
        render(ax, time, threshold, do_threshold)
else:
    display(
        PyplotRenderer(
            lambda ax, frame: render(ax, frame / 10),
            frame_count=20 * 10,
            width=800,
            height=800,
        )
    )

In [ ]:
def get_circular_kernel(width, height=None):
    if height is None:
        height = width
    x_vals = np.linspace(-1, 1, width)
    y_vals = np.linspace(-1, 1, height)
    x_grid, y_grid = np.meshgrid(x_vals, y_vals)
    z_grid = x_grid ** 2 + y_grid ** 2
    mask = (z_grid > 0.7) & (z_grid <= 1)
    z_grid = np.where(mask, 1., 0.)
    z_grid /= z_grid.sum()
    return z_grid

#kernel1 = get_circular_kernel(15)
#kernel2 = get_circular_kernel(7)
#kernel3 = np.random.uniform(0, 1, [7, 7])
#kernel3 /= kernel3.sum()


MAX_ITERATIONS = 10
FRAMES_PER_ITERATION = 5

np.random.seed(37)
mat = np.zeros((60, 60))
mat[22, 7] = 1.
mat[38, 29] = 1.
mat[24, 27] = 1.
mats = [mat]
for i in range(MAX_ITERATIONS):
    kernel = np.random.uniform(0, 1, [9, 9])
    kernel = np.where(kernel > 0.95, 1, 0)
    mat = mat + convolve(mat, kernel, 'same')
    mat = np.where(mat > 1., 1., mat)
    mats.append(mat)


def render(ax, time):
    iteration = int(time) + 1
    interp = time % 1.
    mat1 = mats[iteration - 1]
    mat2 = mats[iteration]
    mat = mat2 * interp + mat1 * (1 - interp)
    ax.imshow(mat, vmin=0, vmax=1, cmap='bone')


if False:
    @interact(
        time=(0., 100., 0.1),
        threshold=(0., 1., 0.01),
    )
    def run(
        time=0.,
        threshold=0.21,
        do_threshold=True,
    ):
        fig = plt.figure(figsize=(14, 10))
        ax = fig.gca()
        render(ax, time, threshold, do_threshold)
else:
    display(
        PyplotRenderer(
            lambda ax, frame: render(ax, frame / FRAMES_PER_ITERATION),
            frame_count=MAX_ITERATIONS * FRAMES_PER_ITERATION,
            width=800,
            height=800,
        )
    )